In [1]:
!pip install ta-lib

In [2]:
# Lấy dữ liệu từ nguồn sẵn có
import sys
import os
import talib as ta
sys.path.append('d:\\NCKH\\SARSA_FinancialRL')  # Thêm đường dẫn gốc để import module

import pandas as pd
from data.data_processor.feature_engineer import engineer_stat as es

# Định nghĩa đường dẫn gốc
base_path = 'd:\\NCKH\\SARSA_FinancialRL'

# Lấy dữ liệu tạm từ nguồn sẵn có
csv_path = os.path.join(base_path, 'data', 'data_storer', 'FPT_detail_2013_01_01_2024_12_31.csv')
df = pd.read_csv(csv_path)

In [3]:
df

,time,open,high,low,close,volume
0,20-06-2012,5.31,5.31,5.27,5.29,89720
1,21-06-2012,5.28,5.29,5.26,5.28,50360
2,22-06-2012,5.27,5.28,5.26,5.26,67350
3,25-06-2012,5.25,5.29,5.25,5.26,42840
4,26-06-2012,5.22,5.24,5.22,5.22,31110
...,...,...,...,...,...,...
3128,25-12-2024,129.85,130.37,129.68,130.02,2517969
3129,26-12-2024,130.19,130.54,128.56,129.16,2238343
3130,27-12-2024,128.90,129.25,128.21,128.99,2567003
3131,30-12-2024,128.81,130.02,128.04,129.76,1986819


### Gắn vào các chỉ báo kĩ thuật cần thiết

In [4]:
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)

# Gọi hàm với start_date và config tùy chỉnh
start_date = pd.to_datetime('01-01-2013', format='%d-%m-%Y')
df_with_indicators = es.add_technical_indicators(
    df, 
    start_date=start_date
)
# df_with_indicators giờ đã có các cột chỉ số, gắn vào df gốc của bạn
df_with_indicators

C:\Users\Admin\AppData\Local\Temp\ipykernel_24920\2641122636.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['time'] = pd.to_datetime(df['time'])


,time,open,high,low,close,volume,MACD,RSI,CCI,ADX
0,02/01/2013,4.00,4.15,4.00,4.13,380320,0.062248,72.002503,253.212693,26.054096
1,03/01/2013,4.13,4.15,4.00,4.07,366960,0.073297,66.503579,180.086580,28.047868
2,04/01/2013,4.05,4.13,4.04,4.13,278020,0.085905,69.049162,156.238566,29.899227
3,07/01/2013,4.11,4.14,4.08,4.11,204570,0.093209,67.215490,133.887463,31.676318
4,08/01/2013,4.12,4.31,4.11,4.31,535860,0.113823,74.506365,161.115250,34.114683
...,...,...,...,...,...,...,...,...,...,...
2991,25/12/2024,129.85,130.37,129.68,130.02,2517969,2.858353,67.076480,80.770363,34.110079
2992,26/12/2024,130.19,130.54,128.56,129.16,2238343,2.718885,63.212308,61.673685,33.596240
2993,27/12/2024,128.90,129.25,128.21,128.99,2567003,2.565070,62.446483,34.093919,32.865699
2994,30/12/2024,128.81,130.02,128.04,129.76,1986819,2.476752,64.541897,51.584632,32.527327


## Kiểm chứng độ chính xác của luồng xử lý dữ liệu và các chỉ báo kỹ thuật được thêm vào dataframe

Từ dữ liệu mẫu đã có trong biến df thực hiện tính thủ công để so sánh với kết quả tính được từ hàm đóng gói trên.

In [5]:
# Tính thủ công các chỉ báo kỹ thuật để kiểm chứng
import numpy as np

# Sao chép df để tính thủ công
df_manual = df.copy()

# Tính MACD
df_manual['MACD'] = ta.MACD(df_manual['close'], fastperiod=12, slowperiod=26, signalperiod=9)[0]

# Tính RSI
df_manual['RSI'] = ta.RSI(df_manual['close'], timeperiod=14)

# Tính CCI
df_manual['CCI'] = ta.CCI(df_manual['high'], df_manual['low'], df_manual['close'], timeperiod=20)

# Tính ADX
df_manual['ADX'] = ta.ADX(df_manual['high'], df_manual['low'], df_manual['close'], timeperiod=14)

# Loại bỏ NaN
df_manual = df_manual.dropna().reset_index(drop=True)

# Lọc từ start_date trở đi
df_manual = df_manual[df_manual['time'] >= start_date].reset_index(drop=True)

# Định dạng time thành dd/mm/yyyy
df_manual['time'] = df_manual['time'].dt.strftime('%d/%m/%Y')

# Giữ lại các cột cần thiết
required_columns = ['time', 'open', 'high', 'low', 'close', 'volume', 'MACD', 'RSI', 'CCI', 'ADX']
df_manual = df_manual[required_columns]

# So sánh với df_with_indicators
print("Shape df_manual:", df_manual.shape)
print("Shape df_with_indicators:", df_with_indicators.shape)

# Kiểm tra các cột giống nhau
columns_match = df_manual.columns.tolist() == df_with_indicators.columns.tolist()
print("Cột giống nhau:", columns_match)

if columns_match:
    # So sánh từng cột numeric
    numeric_cols = ['open', 'high', 'low', 'close', 'volume', 'MACD', 'RSI', 'CCI', 'ADX']
    for col in numeric_cols:
        close_match = np.allclose(df_manual[col], df_with_indicators[col], rtol=1e-10, atol=1e-10)
        print(f"Cột {col} khớp:", close_match)
        if not close_match:
            print(f"  Sai lệch max trong {col}:", np.max(np.abs(df_manual[col] - df_with_indicators[col])))

    # So sánh cột time
    time_match = (df_manual['time'] == df_with_indicators['time']).all()
    print("Cột time khớp:", time_match)

print("\nĐầu df_manual:")
print(df_manual.head())

print("\nĐầu df_with_indicators:")
print(df_with_indicators.head())

Shape df_manual: (2996, 10)
Shape df_with_indicators: (2996, 10)
Cột giống nhau: True
Cột open khớp: True
Cột high khớp: True
Cột low khớp: True
Cột close khớp: True
Cột volume khớp: True
Cột MACD khớp: True
Cột RSI khớp: True
Cột CCI khớp: True
Cột ADX khớp: True
Cột time khớp: True

Đầu df_manual:
         time  open  high   low  close  volume      MACD        RSI  \
0  02/01/2013  4.00  4.15  4.00   4.13  380320  0.062248  72.002503   
1  03/01/2013  4.13  4.15  4.00   4.07  366960  0.073297  66.503579   
2  04/01/2013  4.05  4.13  4.04   4.13  278020  0.085905  69.049162   
3  07/01/2013  4.11  4.14  4.08   4.11  204570  0.093209  67.215490   
4  08/01/2013  4.12  4.31  4.11   4.31  535860  0.113823  74.506365   

          CCI        ADX  
0  253.212693  26.054096  
1  180.086580  28.047868  
2  156.238566  29.899227  
3  133.887463  31.676318  
4  161.115250  34.114683  

Đầu df_with_indicators:
         time  open  high   low  close  volume      MACD        RSI  \
0  02/01/2013 

In [6]:
df_manual

,time,open,high,low,close,volume,MACD,RSI,CCI,ADX
0,02/01/2013,4.00,4.15,4.00,4.13,380320,0.062248,72.002503,253.212693,26.054096
1,03/01/2013,4.13,4.15,4.00,4.07,366960,0.073297,66.503579,180.086580,28.047868
2,04/01/2013,4.05,4.13,4.04,4.13,278020,0.085905,69.049162,156.238566,29.899227
3,07/01/2013,4.11,4.14,4.08,4.11,204570,0.093209,67.215490,133.887463,31.676318
4,08/01/2013,4.12,4.31,4.11,4.31,535860,0.113823,74.506365,161.115250,34.114683
...,...,...,...,...,...,...,...,...,...,...
2991,25/12/2024,129.85,130.37,129.68,130.02,2517969,2.858353,67.076480,80.770363,34.110079
2992,26/12/2024,130.19,130.54,128.56,129.16,2238343,2.718885,63.212308,61.673685,33.596240
2993,27/12/2024,128.90,129.25,128.21,128.99,2567003,2.565070,62.446483,34.093919,32.865699
2994,30/12/2024,128.81,130.02,128.04,129.76,1986819,2.476752,64.541897,51.584632,32.527327


In [7]:
df_with_indicators

,time,open,high,low,close,volume,MACD,RSI,CCI,ADX
0,02/01/2013,4.00,4.15,4.00,4.13,380320,0.062248,72.002503,253.212693,26.054096
1,03/01/2013,4.13,4.15,4.00,4.07,366960,0.073297,66.503579,180.086580,28.047868
2,04/01/2013,4.05,4.13,4.04,4.13,278020,0.085905,69.049162,156.238566,29.899227
3,07/01/2013,4.11,4.14,4.08,4.11,204570,0.093209,67.215490,133.887463,31.676318
4,08/01/2013,4.12,4.31,4.11,4.31,535860,0.113823,74.506365,161.115250,34.114683
...,...,...,...,...,...,...,...,...,...,...
2991,25/12/2024,129.85,130.37,129.68,130.02,2517969,2.858353,67.076480,80.770363,34.110079
2992,26/12/2024,130.19,130.54,128.56,129.16,2238343,2.718885,63.212308,61.673685,33.596240
2993,27/12/2024,128.90,129.25,128.21,128.99,2567003,2.565070,62.446483,34.093919,32.865699
2994,30/12/2024,128.81,130.02,128.04,129.76,1986819,2.476752,64.541897,51.584632,32.527327


## Thử nghiệm hàm lấy dữ liệu từ VNStockDataProvider.py có thể chỉnh biến mình mong muốn

In [8]:
# Thử nghiệm VNStockDataProvider
import sys
sys.path.append('d:\\NCKH\\SARSA_FinancialRL')

from data.data_provider.library_extracted.vnstock.VNStockDataProvider import VNStockDataProvider

# Khởi tạo VNStockDataProvider
provider = VNStockDataProvider()

# Lấy dữ liệu OHLCV cho một số mã cổ phiếu
symbols = ['FPT', 'VCB', 'VNM']
start_date = '2020-01-01'
end_date = '2020-12-31'
interval = '1D'

# Gọi hàm get_ohlcv_data
ohlcv_data = provider.get_ohlcv_data(symbols, start_date, end_date, interval, verbose=True)

ohlcv_data
# # Hiển thị kết quả
# print("Dữ liệu OHLCV:")
# print(ohlcv_data.head(10))
# print(f"\nShape: {ohlcv_data.shape}")
# print(f"Các cột: {ohlcv_data.columns.tolist()}")

# # Kiểm tra cột 'time' có phải là yyyy/mm/dd không
# if 'time' in ohlcv_data.columns:
#     print(f"\nMẫu cột time: {ohlcv_data['time'].head()}")
#     print(f"Format time: {ohlcv_data['time'].dtype}")

LẤY DỮ LIỆU OHLCV TỪ VNSTOCK
📋 Danh sách mã: ['FPT', 'VCB', 'VNM']
📅 Khoảng thời gian: 2020-01-01 đến 2020-12-31
⏱️  Phiên: 1D
📡 Nguồn: VCI
--------------------------------------------------------------------------------
📡 Đang lấy dữ liệu cho FPT...
✅ FPT: 264 dòng
📡 Đang lấy dữ liệu cho VCB...
✅ FPT: 264 dòng
📡 Đang lấy dữ liệu cho VCB...
✅ VCB: 264 dòng
📡 Đang lấy dữ liệu cho VNM...
✅ VCB: 264 dòng
📡 Đang lấy dữ liệu cho VNM...
✅ VNM: 264 dòng

KẾT QUẢ
✅ Tổng số dòng dữ liệu: 792
✅ Số mã thành công: 3

📋 Các cột dữ liệu:
   • date
   • open
   • high
   • low
   • close
   • volume
   • symbol

📊 Thống kê:
   • Số mã: 3
   • Khoảng thời gian: 2019-12-16 đến 2020-12-31

📄 Mẫu dữ liệu (10 dòng đầu):
         date   open   high    low  close   volume symbol
0  2019/12/16  20.84  20.84  20.59  20.63   968130    FPT
1  2019/12/17  20.52  20.59  20.11  20.22  2211360    FPT
2  2019/12/18  20.22  20.44  20.08  20.26  1022490    FPT
3  2019/12/19  20.26  20.52  20.22  20.37   817820    FPT


,date,open,high,low,close,volume,symbol
0,2019/12/16,20.84,20.84,20.59,20.63,968130,FPT
1,2019/12/17,20.52,20.59,20.11,20.22,2211360,FPT
2,2019/12/18,20.22,20.44,20.08,20.26,1022490,FPT
3,2019/12/19,20.26,20.52,20.22,20.37,817820,FPT
4,2019/12/20,20.44,20.70,20.37,20.63,777100,FPT
...,...,...,...,...,...,...,...
787,2020/12/25,82.01,82.23,81.48,82.16,1755510,VNM
788,2020/12/28,82.16,82.68,81.63,81.93,2305300,VNM
789,2020/12/29,82.16,82.46,81.86,82.31,1299700,VNM
790,2020/12/30,82.31,83.06,81.86,82.01,1856600,VNM


In [9]:
ohlcv_data.loc[ohlcv_data['symbol'] == 'FPT']

,date,open,high,low,close,volume,symbol
0,2019/12/16,20.84,20.84,20.59,20.63,968130,FPT
1,2019/12/17,20.52,20.59,20.11,20.22,2211360,FPT
2,2019/12/18,20.22,20.44,20.08,20.26,1022490,FPT
3,2019/12/19,20.26,20.52,20.22,20.37,817820,FPT
4,2019/12/20,20.44,20.70,20.37,20.63,777100,FPT
...,...,...,...,...,...,...,...
259,2020/12/25,24.90,25.16,24.72,25.16,1625110,FPT
260,2020/12/28,25.29,25.60,25.12,25.29,2197090,FPT
261,2020/12/29,25.29,25.68,25.16,25.60,2172950,FPT
262,2020/12/30,25.73,25.73,25.38,25.38,1204180,FPT


In [10]:
# Test VNStockDataProvider
from data.data_provider.library_extracted.vnstock.VNStockDataProvider import VNStockDataProvider

# Khởi tạo provider
provider = VNStockDataProvider(source='VCI')

# Lấy dữ liệu cho một số mã
symbols = ['FPT', 'ACB']
start_date = '2023-01-01'
end_date = '2023-12-31'
interval = '1D'

ohlcv_data = provider.get_ohlcv_data(symbols, start_date, end_date, interval, verbose=True)

print(f"\nColumns: {ohlcv_data.columns.tolist()}")
print(f"Shape: {ohlcv_data.shape}")
print(f"Sample data:\n{ohlcv_data.head()}")

LẤY DỮ LIỆU OHLCV TỪ VNSTOCK
📋 Danh sách mã: ['FPT', 'ACB']
📅 Khoảng thời gian: 2023-01-01 đến 2023-12-31
⏱️  Phiên: 1D
📡 Nguồn: VCI
--------------------------------------------------------------------------------
📡 Đang lấy dữ liệu cho FPT...
✅ FPT: 262 dòng
📡 Đang lấy dữ liệu cho ACB...
✅ FPT: 262 dòng
📡 Đang lấy dữ liệu cho ACB...
✅ ACB: 262 dòng

KẾT QUẢ
✅ Tổng số dòng dữ liệu: 524
✅ Số mã thành công: 2

📋 Các cột dữ liệu:
   • date
   • open
   • high
   • low
   • close
   • volume
   • symbol

📊 Thống kê:
   • Số mã: 2
   • Khoảng thời gian: 2022-12-14 đến 2023-12-29

📄 Mẫu dữ liệu (10 dòng đầu):
         date   open   high    low  close   volume symbol
0  2022/12/14  13.54  13.75  13.34  13.57  3294738    ACB
1  2022/12/15  13.48  13.78  13.48  13.60  4116001    ACB
2  2022/12/16  13.48  13.72  13.40  13.48  2221946    ACB
3  2022/12/19  13.42  13.63  13.34  13.34  3384971    ACB
4  2022/12/20  13.34  13.42  13.01  13.22  2569365    ACB
5  2022/12/21  13.25  13.48  13.07  13.37

In [11]:
# Lưu dữ liệu từ VNStock thành CSV (giá VND đầy đủ, không chuẩn hóa)
if not ohlcv_data.empty:
    output_csv_path = os.path.join(base_path, 'data', 'data_storer', 'data_research', 'vnstock_raw_FPT_ACB_2023.csv')
    ohlcv_data.to_csv(output_csv_path, index=False)
    print(f"Đã lưu dữ liệu thô từ VNStock vào: {output_csv_path}")
    print(f"Shape: {ohlcv_data.shape}")
    print("Mẫu dữ liệu:")
    print(ohlcv_data.head())
else:
    print("Không có dữ liệu để lưu!")

Đã lưu dữ liệu thô từ VNStock vào: d:\NCKH\SARSA_FinancialRL\data\data_storer\data_research\vnstock_raw_FPT_ACB_2023.csv
Shape: (524, 7)
Mẫu dữ liệu:
         date   open   high    low  close   volume symbol
0  2022/12/14  13.54  13.75  13.34  13.57  3294738    ACB
1  2022/12/15  13.48  13.78  13.48  13.60  4116001    ACB
2  2022/12/16  13.48  13.72  13.40  13.48  2221946    ACB
3  2022/12/19  13.42  13.63  13.34  13.34  3384971    ACB
4  2022/12/20  13.34  13.42  13.01  13.22  2569365    ACB
